In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Part 1

In [207]:
import re

class Sensor:
    def _parse_line(self,line):
        str_format = 'x=(-?\d+), y=(-?\d+)'
        sensor, beacon = re.findall(str_format, line)
        sensor = (int(sensor[0]), int(sensor[1]) )
        beacon = (int(beacon[0]), int(beacon[1]) )
        return sensor, beacon
        
    def __init__(self, line):
        line=line.strip()
        self.sensor, self.beacon = self._parse_line(line) 
        self.distance  = (abs(self.sensor[0]-self.beacon[0])
                         +abs(self.sensor[1]-self.beacon[1]))
        
    def blocked_points_for_row(self, row):
        
        d = abs(self.sensor[1] - row)
        if d > self.distance:
            return []
        xvals = [self.sensor[0]]
        if d == 0 :
            return xvals
        
        for i in range(1,self.distance -d +1 ):
            xvals.extend([self.sensor[0]-i,self.sensor[0]+i])
        return xvals
    

    def capped_blocked_interval(self, row, cap):
        d = abs(self.sensor[1] - row)
        if d > self.distance:
            return []
        diff=self.distance -d
        return [max(self.sensor[0]-diff, 0),min(self.sensor[0]+diff, cap)]
    
    
    def walk_edges(self,cap):
        sx,sy = self.sensor
        check_points =[]
        dist=self.distance
        for i in range(0,dist+1):
            xshift=i
            yshift=dist-i
            new=[(sx+xshift+1, sy+yshift),
                 (sx-xshift-1, sy+yshift),
                 (sx-xshift-1, sy-yshift),
                 (sx+xshift+1, sy-yshift),
                ]
            new = [ p for p in new if (p[0] >=0 and p[0] <=cap and p[1] >=0 and p[1] <=cap)]
            check_points.extend(new)
            
        new = [(sx, sy-dist-1),(sx, sy+dist+1) ]
        new = [ p for p in new if (p[0] >=0 and p[0] <=cap and p[1] >=0 and p[1] <=cap)]
        check_points.extend(new)
        return check_points
            
    def point_in_range(self,pt):
        dist = abs(self.sensor[0]-pt[0])+abs(self.sensor[1]-pt[1])
        if dist <= self.distance:
            return 1
        return 0
        


In [203]:
def part_one(filename, row_number):
    sensors = read_file(filename)
    
    row_vals = []
    beacons = []
    for s in sensors:
        blocked = s.blocked_points_for_row(row_number)
        if s.beacon[1] == row_number:
            beacons.append(s.beacon[1])
        if len(blocked) > 0:
            row_vals.extend(blocked)
            
    row_vals = set(row_vals) - set(beacons)
    return len(row_vals)
 

In [204]:
def read_file(filename):
    sensors = []
    with open(filename, 'r') as f:
        for line in f.readlines():
            sensors.append(Sensor(line))
    return sensors

In [170]:
part_one('input/test_15.txt', 10)

26

In [171]:
%%time
part_one('input/day_15.txt', 2000000)

CPU times: user 1.05 s, sys: 273 ms, total: 1.33 s
Wall time: 1.51 s


4883971

# Part 2

In [205]:
test = read_file('input/test_15.txt')

In [206]:
test[0].walk_edges(20)

[(1, 11),
 (3, 11),
 (0, 12),
 (4, 12),
 (5, 13),
 (6, 14),
 (7, 15),
 (8, 20),
 (8, 16),
 (9, 19),
 (9, 17),
 (10, 18),
 (10, 18),
 (2, 10)]

In [210]:
def part_two(filename,cap):
    sensors = read_file(filename)
    for s in sensors:
        check_pts = s.walk_edges(cap)
        for pt in check_pts:
            within_range = sum([ ns.point_in_range(pt) for ns in sensors])
            if within_range == 0:
                return  4000000*pt[0]+pt[1]
        
    
    

In [209]:
%%time
part_two('input/test_15.txt', 20)

56000011
CPU times: user 2.54 ms, sys: 2.59 ms, total: 5.12 ms
Wall time: 3.29 ms


(14, 11)

In [214]:
%%time
part_two('input/day_15.txt', 4e6)

CPU times: user 21.2 s, sys: 282 ms, total: 21.5 s
Wall time: 21.6 s


12691026767556